# Assignement (segmenting and clustering)

In [1]:
import pandas as pd

## Get dataframe from wikipedia

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# import data as dataframe with pandas and select the first table (index 0)
df_toronto = pd.read_html(link)[0]
df_toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Clean dataframe

In [4]:
# rename postalcode column
df_toronto.rename(columns={'Postcode': 'PostalCode'}, inplace=True)

In [5]:
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
#drop cells with "not-assined" borough
df_toronto = df_toronto[df_toronto['Borough']!='Not assigned']
df_toronto.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


## Format dataframe

In [7]:
#combine rows with same postal code and borough
df_toronto_merged = df_toronto.groupby(['PostalCode', 'Borough'], sort=False)

#merge neighbourhood names in same row
df_toronto_merged = df_toronto_merged['Neighbourhood'].apply(','.join).reset_index()

In [8]:
df_toronto_merged.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [9]:
#check for not assigned neighbourhood
df_toronto_merged[df_toronto_merged.Neighbourhood == 'Not assigned']

,PostalCode,Borough,Neighbourhood
5,M9A,Queen's Park,Not assigned


In [10]:
#get indexes for not assigned neighbourhood
i_not_assigned = df_toronto_merged[df_toronto_merged.Neighbourhood == 'Not assigned'].index

In [11]:
#assign name of borough to not assigned neighbourhood
df_toronto_merged.loc[i_not_assigned, 'Neighbourhood'] = df_toronto_merged['Borough']

In [12]:
#checking all neighbourhood are assigned
df_toronto_merged[df_toronto_merged.Neighbourhood == 'Not assigned']

,PostalCode,Borough,Neighbourhood


In [13]:
df_toronto_merged.shape

(103, 3)

## Get location data

We will use the Geocoder Python package (https://geocoder.readthedocs.io/index.html)

### update: the geocoder didn't work (keep returning None), so location extracted from csv file below

In [14]:
#installing the package 
!pip install geocoder

In [15]:
import geocoder

In [16]:
df_toronto_merged.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


In [17]:
#getting list of all postal code
pc_list = df_toronto_merged['PostalCode'].to_list()

## Get location data from csv

In [18]:
csv_link = 'http://cocl.us/Geospatial_data'

In [19]:
df_loc = pd.read_csv(csv_link)
df_loc

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


First, we will rename the Postal Code column from the location dataframe to have the same name as the Toronto dataframe. 

Then, we will merge the two dataframes based on the column 'PostalCode'

In [20]:
df_loc.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

In [21]:
df_toronto_final = pd.merge(df_toronto_merged, df_loc, on=['PostalCode'])

In [22]:
df_toronto_final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


## Explore the neighbourhoods

In [42]:
import requests

Credentials to connect to foursquare API to explore neighbourhoods

In [24]:
CLIENT_ID = 'xxx' # your Foursquare ID
CLIENT_SECRET = 'xxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [36]:
#get a list of the neighbourhood
neigh = df_toronto_final['Neighbourhood']

### Explore the venues in Toronto using the function created in the past lab

We will use this function to get the categories of the venues

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
#set a limit of 100
LIMIT = 100

In [43]:
toronto_venues = getNearbyVenues(names=df_toronto_final['Neighbourhood'],
                                   latitudes=df_toronto_final['Latitude'],
                                   longitudes=df_toronto_final['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights,Lawrence Manor
Queen's Park
Queen's Park
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design Exchange,Toronto 

In [44]:
#print head of the venues
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


## Analyze each neighborhood

In [49]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#### Get weighted categories per neighborhood

In [51]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.01
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00
4,"Alderwood,Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00


## Clustering the neighborhoods

In [54]:
from sklearn.cluster import KMeans 

We choose to cluster the neighborhoods into 5 clusters

In [55]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 4])

#### Add back the cluster number to each venue

In [58]:
toronto_grouped['cluster'] = kmeans.labels_

In [62]:
toronto_data = toronto_grouped[['Neighborhood', 'cluster']]
toronto_data.rename(columns={'Neighborhood': 'Neighbourhood'}, inplace=True)

c:\users\roman\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [64]:
toronto_clust = pd.merge(toronto_data, df_toronto_final, on=['Neighbourhood'])

In [65]:
toronto_clust.head()

,Neighbourhood,cluster,PostalCode,Borough,Latitude,Longitude
0,"Adelaide,King,Richmond",0,M5H,Downtown Toronto,43.650571,-79.384568
1,Agincourt,0,M1S,Scarborough,43.794200,-79.262029
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",1,M1V,Scarborough,43.815252,-79.284577
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0,M9V,Etobicoke,43.739416,-79.588437
4,"Alderwood,Long Branch",0,M8W,Etobicoke,43.602414,-79.543484


## Visualize clusters

In [70]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

latitude=43.650571
longitude=-79.384568

In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clust['Latitude'], toronto_clust['Longitude'], toronto_clust['Neighbourhood'], toronto_clust['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters